Anne Katrine Falk 11OCT2021
# Animation with matplotlib
This notebook demonstrates how to 

- create plots with animations - with three ways of generating the frames 
- save the animations
- display animation as web content (html5)

In [1]:
# Make cell width follow the width of the window
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important;}</style>"))

# Import dependencies

In [2]:
from datetime import datetime
# animations don't show when using inline or notebook
#%matplotlib inline
#%matplotlib notebook
%matplotlib qt
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import os

print("matplotlib version: " + matplotlib.__version__)
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

matplotlib version: 3.3.2
numpy version: 1.20.2
pandas version: 1.1.4


# Generate some data for animation
Most tutorials I have seen use on-the-fly-generated data. This way, they can obtain a light-weight demonstration, where the "incoming" data is generated as a part of the function which updates the figure. For visualising progressing forecasts we need a flow that _gets the data from somewhere else_. So we start by creating some data, which is persisted in (csv)-files.
The data emulates a 24h forecast, generated once an hour. The forecast is a sine, with some random noise added. Each csv-file includes 24 time steps.

In [3]:
def make_noisy_sine():
    """
    Make one sine wave with a period of 24, and add random noise
    """
    x = np.arange(0,24)
    clean_sin = np.sin(2.*np.pi/24.*x)
    noisy_sin = clean_sin + (np.random.rand(24)-0.5)
    return noisy_sin

In [4]:
def generate_data_files(n):
    """
    For each of the n forecasts:
        create a dataframe with time stamps as index and noisy_sine as values and save as csv.
    Time step in data file is one hour.
    Time between forecasts is one hour.
    """
    start_date = pd.Timestamp('2021-10-10 00')
    for i in range(n):
        data = make_noisy_sine()
        index = pd.date_range(start=start_date, periods = len(data), freq='H')
        df = pd.DataFrame(index=index, data=data, columns=['values'])  
        file_name = start_date.strftime("%Y-%m-%d_%H")
        file_path = os.path.join(".", "data", f"{file_name}.csv")
        df.to_csv(file_path)
        
        start_date = start_date + pd.Timedelta('1H')

Create the data files

In [5]:
generate_data_files(24)

# Method to read data

In [6]:
def get_data(start_date):
    """
    Read a file, which is named by the time stamp of its first value and return a DataFrame
    """
    start_date = pd.Timestamp(start_date)
    file_name = start_date.strftime("%Y-%m-%d_%H")
    file_path = os.path.join(".", "data", f"{file_name}.csv")
    df = pd.read_csv(file_path, index_col=0, parse_dates=True)
    return df

In [7]:
def get_file_path(start_date):
    """
    Read a file, which is named by the time stamp of its first value and return a DataFrame
    """
    start_date = pd.Timestamp(start_date)
    file_name = start_date.strftime("%Y-%m-%d_%H")
    file_path = os.path.join(".", "data", f"{file_name}.csv")
    return file_path

# The animation part starts here

## Setup plot
The animation consists of an ordinary pyplot plot, where (parts of) the data content is manipulated and then updated in the plot. Run this ceel to create the plot, before running any of the below methods to create the animation.

In [30]:
start = pd.Timestamp('2021-10-10 00:00:00')

# Need fig and ax objects. fig will be passed to FuncAnimation (later) and ax holds the artist objects,
# in this case the line with noisy sine and the text showing the start time of each "simulation"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,3))
# add an empty line to the plot. This is a placeholder for the forecast
ax.plot([],[])
# Pick the line, which has just been added to the plot.
# This variable must be visible to the update function (see below), which will provide new data for it.
line = ax.get_lines()[-1] 

# We also want to update the start time of each simulation. This variable must also be visible to the update function
time_text = ax.text(0.65, 0.92, "                    " , transform=ax.transAxes)

# Need to set the axis ranges, because the initial plot does not see the full extent in neither x nor y-direction
end = start + 2*pd.Timedelta('1D')
ax.set_xlim(start, end)
ax.set_ylim(-2, 2)

fig.autofmt_xdate() # beautify the x-labels

## Animate

### Method 1 - frames identified by an integer

The following two cells use an integer to identify the frame in the update function. So in update(i), the file_path is inferred by calculating the time stamp that corresponds to i, and then constructing the file_path.
In the nex cell FuncAnimation takes a frames argument, which is just a list of integers, frames = range(30). We have 24 files, and the range is set to (0,30) to also encounter "missing" files.

In [ ]:
def update(i):
    time = pd.Timestamp(start) + i*pd.Timedelta('1H')
    
    file_path = get_file_path(time)
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col=0, parse_dates=True)
        line.set_data(df.index, df['values'])
        time_text.set_text(str(time))
    else:
        line.set_data([],[])
        time_text.set_text(str(time) + ' MISSING')
    
    # the update function must return all manipulated Artist objects in a list-like object
    return line, time_text

Create the animation (see matplotlib [documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html) to see all possible arguments).
The call to FuncAnimation starts the animation - but for some reason this does not works in an interactive plot. To see the animation without saving, switch to qt backend in the "import dependencies section"

In [ ]:
ani = animation.FuncAnimation(
    fig, func=update, frames=range(30), interval=400, blit=True)#, save_count=24)
    #fig, func=update, interval=400, blit=True, save_count=24)

### Method 2 - frames identified by time stamp

The next two celss show an alternative way of defining which frames to iterate over. Now the update function takes "time", a pd.Timestamp as argument. FuncAnimation gets for "frames" now a list of pd.Timestamp (generated by pd.date_range) - so no need to calculate the time from i.
frames can be any list of objects (possibly also the data objects themselves or a generator function)

In [ ]:
def update(time):
    #time = pd.Timestamp(start) + i*pd.Timedelta('1H')
    
    file_path = get_file_path(time)
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col=0, parse_dates=True)
        line.set_data(df.index, df['values'])
        time_text.set_text(str(time))
    else:
        line.set_data([],[])
        time_text.set_text(str(time) + ' MISSING')
    
    # the update function must return all manipulated Artist objects in a list-like object
    return line, time_text

In [ ]:
frames = pd.date_range(start=start, periods=30, freq='H')
ani = animation.FuncAnimation(
    fig, func=update, frames=frames, interval=400, blit=True)

### Method 3 - frames created by a generator

In [31]:
def frame_generator(time):
    while True:
        file_path = get_file_path(time)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, index_col=0, parse_dates=True)
            time += pd.Timedelta('1H')
            yield df
        else:
            time += pd.Timedelta('1H')
            yield pd.DataFrame()

In [26]:
# optional: see an item returned by the generator
gen = frame_generator(start)
next(gen)

,values
2021-10-10 00:00:00,0.296488
2021-10-10 01:00:00,0.274095
2021-10-10 02:00:00,0.765325
2021-10-10 03:00:00,0.982158
2021-10-10 04:00:00,1.273929
2021-10-10 05:00:00,0.487219
2021-10-10 06:00:00,1.127626
2021-10-10 07:00:00,0.906366
2021-10-10 08:00:00,0.444090
2021-10-10 09:00:00,0.732562


In [32]:
def update(df):
    if ~(df.empty):
        line.set_data(df.index, df['values'])
        time_text.set_text(df.index[0])
    else:
        line.set_data([],[])
        time_text.set_text(' MISSING')
    
    # the update function must return all manipulated Artist objects in a list-like object
    return line, time_text

In [33]:
ani = animation.FuncAnimation(
    fig, func=update, frames=frame_generator(start), interval=400, blit=True, save_count=24)

## Save the animation to different formats
Documentation of different writer classes [https://matplotlib.org/stable/api/animation_api.html#writer-classes](https://matplotlib.org/stable/api/animation_api.html#writer-classes)

### gif

In [ ]:
movie_name = 'test_ani.gif'
ani.save(movie_name, writer=None) #writer=None: default uses Pillow writer

### mp4
Requires the ffmpeg writer. This can be installed as a part of ImageMagic. ImageMagic is not a python package, it needs to be installed separately (download [here](https://imagemagick.org/script/download.php) or install via [chocolatey](https://chocolatey.org/)) - there are probably also other ways, but this is the one that I could easily make work. ImageMagic puts its bin-directory in the PATH environment variable, but you may need to tell matplotlib the name of the executable. This is done in rcParams (se below cell). Just to clarify before entering anything:

- path to bin (installation default): 'C:\Program Files\ImageMagick-7.1.0-Q16-HDRI' (this is what in in PATH)
- name of executable: 'ffmpeg.exe'

In [ ]:
# These two ways of setting the path to the ffmpeg writer both works - the latter is the most "clean".
#plt.rcParams['animation.ffmpeg_path'] = r'C:\Program Files\ImageMagick-7.1.0-Q16-HDRI\ffmpeg.exe' # full path to ffmpeg executable
plt.rcParams['animation.ffmpeg_path'] = 'ffmpeg' #should be specific enough when the PATH includes ffmpeg's bin directory
movie_name = 'test_ani.mp4'
ani.save(movie_name, writer='ffmpeg')

### html5

In [ ]:
# generate a html5 video tag, which can be embedded on a web page
html5_video = ani.to_html5_video()

In [ ]:
# show the generated video tag
html5_video

In [ ]:
# html can also be embedded in the notebook - so show the video!
HTML(html5_video)

# Bonus - a cool way of using animation in combination with polar coordinates :-)

In [ ]:
#https://python.plainenglish.io/building-an-analog-clock-using-python-518922d57784
from numpy import pi
fig= plt.figure(figsize=(2.5,2.5),dpi=100)
ax = fig.add_subplot(111, polar=True)
def update(now):
    plt.cla()
    
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.set_xticks(np.linspace(0, 2*pi, 12, endpoint=False))
    ax.set_xticklabels(range(1,13))
    ax.set_theta_direction(-1)
    ax.set_theta_offset(pi/3.0)
    ax.grid(False)
    plt.ylim(0,1)
    now = datetime.now()
    hour= now.hour
    minute = now.minute
    second = now.second
    angles_h =2*pi*hour/12+2*pi*minute/(12*60)+2*second/(12*60*60)-pi/6.0
    angles_m= 2*pi*minute/60+2*pi*second/(60*60)-pi/6.0
    angles_s =2*pi*second/60-pi/6.0
    ax.plot([angles_s,angles_s], [0,0.9], color="black", linewidth=1)
    ax.plot([angles_m,angles_m], [0,0.7], color="black", linewidth=2)
    ax.plot([angles_h,angles_h], [0,0.3], color="black", linewidth=4)
    return ax
ani = animation.FuncAnimation(fig,update, interval = 100)
plt.show()